In [24]:
import os
import json
import requests
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_community.utilities import OpenWeatherMapAPIWrapper
from typing import TypedDict
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from langgraph.graph import StateGraph, END, START

load_dotenv()

TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")
TYPHOON_KEY = os.getenv("TYPHOON_KEY")
WEATHER_KEY = os.getenv("WEATHER_KEY")

In [ ]:
llm = ChatOpenAI(
    base_url="https://api.opentyphoon.ai/v1",
    api_key=TYPHOON_KEY,
    model='typhoon-v2.5-30b-a3b-instruct',
    temperature=0.5,
    max_tokens=10000,
    streaming=True
)

weather = OpenWeatherMapAPIWrapper(
    openweathermap_api_key=WEATHER_KEY
)


class State(TypedDict):
    weather: str
    question: str
    city: str
    preferences: str
    rag_places: str
    travel_dates: str
    final: str
    search: str


def extract_node(state: State):
    prompt = f"""
        วิเคราะห์คำถามของผู้ใช้งานเพื่อสกัดข้อมูลการท่องเที่ยวออกมาเป็น JSON Format

        User question:
        {state['question']}

        กฎ: 
        1. สกัดชื่อเมือง จังหวัด หรือสถานที่หลักออกมา
        2. แปลงชื่อเมืองนั้นเป็น "ภาษาอังกฤษ" ให้ถูกต้องตามมาตรฐานสากล (เช่น "เขาใหญ่" -> "Khao Yai", "เชียงใหม่" -> "Chiang Mai")
        3. สกัดสไตล์การเที่ยว (เช่น คาเฟ่, ทะเล, ภูเขา, ธรรมชาติ) ถ้าไม่มีให้ใส่ "ทั่วไป"

        ตัวอย่าง JSON Format ที่ต้องการ:
        {{
            "city": "ชื่อเมืองภาษาอังกฤษ",
            "preferences": "สไตล์การเที่ยว"
            "travel_dates": "วันที่เดินทาง เช่น 3-6 มีนาคม"
        }}
        ตอบเฉพาะ JSON เท่านั้น ห้ามมีคำอธิบายอื่น
    """

    response = llm.invoke(prompt).content.strip()

    try:
        if response.startswith("```json"):
            response = response[7:-3].strip()
        elif response.startswith("```"):
            response = response[3:-3].strip()

        data = json.loads(response)
        city = data.get("city", "")
        prefs = data.get("preferences", "ทั่วไป")
        dates = data.get("travel_dates", "ไม่ระบุ")
    except json.JSONDecodeError:
        city = response
        prefs = "ทั่วไป"
        dates = "ไม่ระบุ"

    return {"city": city, "preferences": prefs, "travel_dates": dates}


def search_node(state: State):
    search = TavilySearchResults(max_results=3)
    query = f"สถานที่ท่องเที่ยว {state.get('city', '')} สไตล์ {state.get('preferences', '')}"
    docs = search.invoke({"query": query})

    return {"search": str(docs)}


def weather_node(state: State):
    city = state.get("city")
    if not city:
        return {"weather": "ไม่พบชื่อสถานที่สำหรับตรวจสอบสภาพอากาศ"}

    try:
        geo_url = f"http://api.openweathermap.org/geo/1.0/direct?q={city}&limit=1&appid={WEATHER_KEY}"
        geo_response = requests.get(geo_url).json()

        if not geo_response or len(geo_response) == 0:
            return {"weather": f"ไม่พบข้อมูลพิกัดสำหรับสถานที่: {city}"}

        lat = geo_response[0]['lat']
        lon = geo_response[0]['lon']
        actual_location_name = geo_response[0].get('name', city)

        weather_url = f"https://api.openweathermap.org/data/2.5/weather?lat={lat}&lon={lon}&appid={WEATHER_KEY}&units=metric&lang=th"
        weather_response = requests.get(weather_url).json()

        if weather_response.get("cod") != 200:
            return {"weather": f"Weather error: {weather_response.get('message')}"}

        desc = weather_response['weather'][0]['description']
        temp = weather_response['main']['temp']
        humidity = weather_response['main']['humidity']

        summary = (
            f"สถานที่: {actual_location_name} (พิกัด: {lat}, {lon})\n"
            f"อุณหภูมิ: {temp}°C\n"
            f"สภาพอากาศ: {desc}\n"
            f"ความชื้น: {humidity}%"
        )

        return {"weather": summary}

    except Exception as e:
        return {"weather": f"System error ในระหว่างดึงข้อมูลสภาพอากาศ: {e}"}


def combine_node(state: State):
    prompt = f"""
        คุณคือสุดยอดผู้ช่วยวางแผนการท่องเที่ยว (Dynamic Travel Planner) ที่จัดทริปได้น่าตื่นตาตื่นใจและใช้งานได้จริง
        หน้าที่ของคุณคือจัดทริปโดยอิงจาก "สภาพอากาศจริง" และ "สถานที่จากระบบ" เท่านั้น

        [คำถามจากผู้ใช้]: {state.get('question')}
        [สไตล์การเที่ยว]: {state.get('preferences')}
        [ข้อมูลสภาพอากาศปัจจุบัน]: {state.get('weather')}
        [สถานที่ท่องเที่ยวที่ค้นพบ]: {state.get('search')}

        กฎเหล็ก (Strict Rules):
        1. ยึดสภาพอากาศเป็นหลัก: ฝนตก/พายุ -> ต้องจัดสถานที่ Indoor | แดดจัด/โปร่ง -> จัด Outdoor ได้
        2. ต้องบอกเหตุผลเสมอว่าทำไมถึงเลือกที่นี่ในสภาพอากาศแบบนี้

        **รูปแบบการแสดงผลที่ต้องการ (Must Output in this format):**
        ให้จัด Layout ให้น่าอ่าน ใช้ Emoji ประกอบ และแบ่งหมวดหมู่ดังนี้:

        ✨ **[ตั้งชื่อทริปสุดเก๋ให้เข้ากับเมืองและสไตล์]**
        ☁️ **สภาพอากาศวันนี้:** (สรุปสภาพอากาศ และอุณหภูมิ)
        🎒 **ไอเทมแนะนำ:** (แนะนำการแต่งตัวหรือของที่ต้องพกให้เข้ากับสภาพอากาศ เช่น พกร่ม, ทาครีมกันแดด, เสื้อกันหนาว)

        🗺️ **แผนการเดินทาง (Itinerary):**
        จัดตารางเวลา (เช้า, เที่ยง, บ่าย) โดยระบุ:
        - **เวลา:** กิจกรรมและชื่อสถานที่
        - **เหตุผล:** (อ้างอิงสภาพอากาศ)
        - **นำทาง:** [คลิกเพื่อดู Google Maps](https://www.google.com/maps/search/?api=1&query=ชื่อสถานที่ภาษาอังกฤษ+เมือง)  <-- (สร้าง URL แบบนี้ให้ทุกสถานที่)

        💡 **Pro Tip:** ทริคเล็กๆ น้อยๆ สำหรับการไปเที่ยวที่นี่ในวันนี้
    """

    collected = ""
    print("\nPlanner Bot:\n")
    for chunk in llm.stream(prompt):
        token = chunk.content
        print(token, end="", flush=True)
        collected += token
    print("\n")

    return {"final": collected}


builder = StateGraph(State)
builder.add_node("extract", extract_node)
builder.add_node("search", search_node)
builder.add_node("weather", weather_node)
builder.add_node("combine", combine_node)

builder.add_edge(START, "extract")

builder.add_edge("extract", "search")
builder.add_edge("extract", "weather")

builder.add_edge("search", "combine")
builder.add_edge("weather", "combine")

builder.add_edge("combine", END)
graph = builder.compile()

if __name__ == "__main__":
    q = input("\nคุณ: ")
    for _ in graph.stream({"question": q}):
        pass


Planner Bot:

✨ **[ทริปชิลๆ ใต้แสงแดดอ่อน บางแสนแบบไม่ต้องกลัวฝน!]**  
☁️ **สภาพอากาศวันนี้:** เมฆกระจาย อุณหภูมิ 27.42°C — แดดไม่จัด ไม่มีฝน ไม่มีพายุ บรรยากาศสบายๆ เหมาะกับการออกไปเที่ยวกลางแจ้งทุกมุม!  
🌤️ ท้องฟ้าเปิดช่วงบ่ายอาจมีแสงแดดอ่อนๆ แต่ยังคงความชื้นสูงเล็กน้อย ควรเตรียมตัวให้พร้อมกันแดดและลมทะเล  

🎒 **ไอเทมแนะนำ:**  
- ครีมกันแดด SPF 50+ (เพราะแม้เมฆกระจาย แต่รังสี UV ก็ยังแรง!)  
- หมวกกันแดดหรือผ้าคลุมศีรษะแบบผ้าบางเบา  
- แว่นกันแดด (เพื่อความปลอดภัยและถ่ายรูปสวยขึ้น)  
- กระเป๋าเป้สะพายข้าง (พกน้ำดื่ม ผ้าขนหนู หรือของกินเล่นได้สะดวก)  
- กล้องถ่ายรูปหรือมือถือพร้อมแบตเตอรี่สำรอง (เพราะมุมถ่ายรูปสวยทุกจุด!)

---

🗺️ **แผนการเดินทาง (Itinerary):**

🌞 **เช้า (08:00 – 11:00)**  
- **เวลา:** 08:00 – 10:00  
- **กิจกรรม:** เดินเล่นริมชายหาดบางแสน + ถ่ายรูปช่วงเช้าที่หาดวอนนภา  
- **สถานที่:** หาดวอนนภา (Wonnapha Beach)  
- **เหตุผล:** ช่วงเช้าอากาศเย็นสบาย ลมทะเลพัดเบาๆ ไม่ร้อนจนเกินไป และแสงธรรมชาติยังนุ่มนวลเหมาะกับการถ่ายรูปแนวชิลๆ หรือภาพวิวทะเลยามเช้าที่เงียบสงบ  
- **